In [ ]:
''' PREREQUISITES:
    - Get an environment variable called GOOGLE_CREDS with the json of credentials
        used to access google API (cf. 1Password)
    - Run pip install all the packages listed in the import section if not already done 
        * pip install google 
        * pip install googleapiclient
'''

In [1]:
''' METHODS:
    - get_dataframe(spreadsheet_id, worksheet_name, range_name, headers=1)
    - write_dataframe(df, spreadsheet_id, worksheet_name, row=1, col=1)
    - clear_worksheet(spreadsheet_id, worksheet_name)
'''

' METHODS:\n    - get_dataframe(spreadsheet_id, worksheet_name, range_name, headers=1)\n    - write_dataframe(df, spreadsheet_id, worksheet_name, row=1, col=1)\n    - clear_worksheet(spreadsheet_id, worksheet_name)\n'

In [1]:
import os.path
import pandas as pd 
import json

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

class GoogleSheetData:
    """ Class that handles data requests about google sheets
    """

    def __init__(self):
        """ Connects to gsheet using env variable GOOGLE_CREDS (1st time connecting)
        or local file token.json if exists (subsequent connections).    
        """

        credentials = None
        # The file token.json stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists("token.json"):
            credentials = Credentials.from_authorized_user_file("token.json", SCOPES)

        # If there are no (valid) credentials available, let the user log in.
        if not credentials or not credentials.valid:
            if credentials and credentials.expired and credentials.refresh_token:
                credentials.refresh(Request())
            else:
                config = json.loads(os.environ['GOOGLE_CREDS'])
                flow = InstalledAppFlow.from_client_config(
                        config, SCOPES)

                credentials = flow.run_local_server(port=0)

            # Save the credentials for the next run
            with open("token.json", "w") as token:
                token.write(credentials.to_json())

        self.creds = credentials
        return print('Connected to GSheets')


    def get_dataframe(self, spreadsheet_id, worksheet_name, range_name, headers=1):
        """ Returns pandas dataframe from a spreadsheet.

            :param spreadsheet_id:    ID of the spreadhsheet, found in its URL: str
            :param worksheet_name:    name of worksheet e.g. "Sheet1": str
            :param range_name:        range to take into account e.g. "A1:E15": str
            :param headers:           whether selection has headers (1) or not (0). Default 1: int

            :return:                  pandas dataframe of the selection: df
        """

        total_range = worksheet_name + '!' + range_name

        try:
            service = build("sheets", "v4", credentials=self.creds)

            # Call the Sheets API
            sheet = service.spreadsheets()
            result = (
                sheet.values()
                .get(spreadsheetId=spreadsheet_id, range=total_range)
                .execute()
            )
            values = result.get("values", [])

            if not values:
                print("No data found.")
                return
            
            if headers > 0:
                df = pd.DataFrame(values[1:], columns=values[0])
            else:
                df = pd.DataFrame(values)

            return df

        except HttpError as err:
            print(err)


    def write_dataframe(self, df, spreadsheet_id, worksheet_name, row=1, col=1):
        """ Writes a dataframe into a google worksheet.

            :param df:                  pandas dataframe to write to GSheet: df
            :param spreadsheet_id:      ID of the spreadhsheet, found in its URL: str
            :param worksheet_name:      name of worksheet, e.g. "Sheet1": str
            :param row:                 row number where df is going to be written, default 1: int
            :param col:                 column number where df is going to be written, default 1: int
            
            :return:                    dict of spreadsheet & range that has been updated, with which cells have been written on: dict
        """

        col_letter_start = chr(ord('@') + col) # Transforms a column nb into a letter, e.g. 2 -> B
        row_number_start = row

        range_name = col_letter_start + str(row_number_start)
        total_range = worksheet_name + '!' + range_name

        try:
            service = build("sheets", "v4", credentials=self.creds)

            col_names = [df.columns.values.tolist()]
            values = col_names + df.values.tolist()

            body = {"values": values}

            result = (
                service.spreadsheets()
                .values()
                .update(
                    spreadsheetId=spreadsheet_id,
                    range=total_range,
                    valueInputOption="USER_ENTERED",
                    body=body,
                )
                .execute()
            )
            return result
        
        except HttpError as error:
            print(f"An error occurred: {error}")
            return error
        

    def clear_worksheet(self, spreadsheet_id, worksheet_name):
        """ Clears data from a worksheet.

            :param spreadsheet_id:      ID of the spreadhsheet, found in its URL: str
            :param worksheet_name:      name of worksheet, e.g. "Sheet1": str

            :return:                    dict of spreadsheet & range that has been cleared: dict
        """

        try:
            service = build("sheets", "v4", credentials=self.creds)

            result = (
                service.spreadsheets()
                .values()
                .clear(
                    spreadsheetId=spreadsheet_id,
                    range=worksheet_name,
                )
                .execute()
            )
            return result
        
        except HttpError as error:
            print(f"An error occurred: {error}")
            return error

In [ ]:
gs = GoogleSheetData()

# Get dataframe 
df = gs.get_dataframe("1R3Bw9aUNvJm-1HxPTYLJyDPT9Vt0mINwQI2bq1HMfxA", "Sheet1", "A1:B100")

# Write dataframe 
gs.write_dataframe(df, "1R3Bw9aUNvJm-1HxPTYLJyDPT9Vt0mINwQI2bq1HMfxA", "Sheet2")

# Clear worksheet 
gs.clear_worksheet("1R3Bw9aUNvJm-1HxPTYLJyDPT9Vt0mINwQI2bq1HMfxA", "Sheet2")

In [ ]:
df = pd.read_csv('/Users/leazurfluh/Downloads/DO NOT EDIT -Data Recon 23 & 24/2023 CSV Files/2023_bumble_marketing_spend_editJan24_no_meta_college_refactored.csv')


In [2]:
path_files_2023 = '/Users/leazurfluh/Downloads/GCP_marketing_spend/2023_archive/'
path_files_2024 = '/Users/leazurfluh/Downloads/GCP_marketing_spend/2024_archive/'
path_original_files_2023 = '/Users/leazurfluh/Downloads/DO NOT EDIT -Data Recon 23 & 24/2023 CSV Files/'
suffix = '_refactored.csv'

In [3]:
df_dict = {
    '2023_bumble_marketing_spend_editJan24_no_meta_college': path_files_2023,
    'Restated & Net New MMM Data - College Ambassador 2023': path_files_2023,
    '2023 Meta Restated': path_files_2023,
    '2024 Meta Restated': path_files_2024,
    'AB Updated MMM H1 2024 US Data - NYT': path_files_2024,
    'AB Updated MMM H1 2024 US Data - Snapchat': path_files_2024,
    'AB Updated MMM H1 2024 US Data - Vox': path_files_2024,
    'AB Updated MMM H1 2024 US Data - YouTube via MiQ': path_files_2024,
    'AB Updated MMM H1 2024 US Data - iHeart': path_files_2024,
    'US_Bumble_SPAN_2024 AP': path_files_2024,
    'US_Bumble_Dating Sunday_January 2024': path_files_2024,
    'US_Bumble_Audio_Q2 2024': path_files_2024,
    'US_Bumble_Anything Goes_Dating Exclusivity_2024 AP': path_files_2024,
    'IAT_Bumble_Q4_US_2024_MM': path_files_2024,
    'Havas_Bumble_Q3_US_2024_MM': path_files_2024,
    'Aug-Dec 2024 Havas Meta': path_files_2024,
    'AB Updated MMM Q3 2024 US Data - TikTok': path_files_2024,
    'Restated & Net New MMM Data - TikTok (1)': path_files_2024,
    'AB Updated MMM H1 2024 US Data - TikTok': path_files_2024,
    'AB Updated MMM H1 2024 US Data - Google (YouTube)': path_files_2024,
    'AB Updated MMM H1 2024 US Data - CTV via MiQ (thru 6_30)': path_files_2024,
    'AB Updated MMM Q3 2024 US Data - YouTube via Miq': path_files_2024,
    'Restated & Net New MMM Data - Meme': path_files_2024,
    'AB Updated MMM Q4 2024 US Data -  Oct OOH': path_files_2024,
    'Restated & Net New MMM Data - Influencer': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - December CTV via MiQ': path_files_2024,
    'AB Updated MMM Q3 2024 US Data - Uber': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Oct YouTube via MiQ': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Oct TikTok': path_files_2024,
    'Restated & Net New MMM Data - City Marketing': path_files_2024,
    'Restated & Net New MMM Data - Pinterest': path_files_2024,
    'AB Updated MMM Q3 2024 US Data - OOH': path_files_2024,
    'AB Updated MMM Q3 2024 US Data - CTV via MiQ': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Nov YouTube via MiQ': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - December YouTube via MiQ': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Nov TikTok': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - December TikTok': path_files_2024,
    'Restated & Net New MMM Data - Culture': path_files_2024,
    'Restated & Net New MMM Data - College Ambassador 2024': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - December Snapchat': path_files_2024,
    'Restated & Net New MMM Data - Vox': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Nov CTV via MiQ': path_files_2024,
    'Restated & Net New MMM Data - OOH': path_files_2024,
    'Restated & Net New MMM Data - Live Nation (OOH)': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - December OOH': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Nov OOH': path_files_2024,
    'Net New 2024 MMM Spend - Net New Items': path_files_2024,
}

In [11]:
df1 = pd.read_csv(path_original_files_2023 + '2023_bumble_marketing_spend_editJan24_no_meta_college.csv')
df3 = pd.read_csv(path_original_files_2023 + 'Restated & Net New MMM Data - College Ambassador 2023.csv')
df9 = pd.read_csv(path_original_files_2023 + '2023 Meta Restated.csv')

/var/folders/6s/c8mm3wnn6s51d7sp8cgj057c0000gp/T/ipykernel_2280/1520204487.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(path_original_files_2023 + '2023_bumble_marketing_spend_editJan24_no_meta_college.csv')


In [5]:
df1 = pd.read_csv(path_files_2023 + '2023_bumble_marketing_spend_editJan24_no_meta_college' + suffix)
df2 = pd.read_csv(path_files_2024 + '2024 Meta Restated' + suffix)
df3 = pd.read_csv(path_files_2023 + 'Restated & Net New MMM Data - College Ambassador 2023' + suffix)
df4 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - NYT' + suffix)
df5 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - Snapchat' + suffix)
df6 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - Vox' + suffix)
df7 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - YouTube via MiQ' + suffix)
df8 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - iHeart' + suffix)
df9 = pd.read_csv(path_files_2023 + '2023 Meta Restated' + suffix)
df10 = pd.read_csv(path_files_2024 + 'US_Bumble_SPAN_2024 AP' + suffix)
df11 = pd.read_csv(path_files_2024 + 'US_Bumble_Dating Sunday_January 2024' + suffix)
df12 = pd.read_csv(path_files_2024 + 'US_Bumble_Audio_Q2 2024' + suffix)
df13 = pd.read_csv(path_files_2024 + 'US_Bumble_Anything Goes_Dating Exclusivity_2024 AP' + suffix)
df14 = pd.read_csv(path_files_2024 + 'IAT_Bumble_Q4_US_2024_MM' + suffix)
df15 = pd.read_csv(path_files_2024 + 'Havas_Bumble_Q3_US_2024_MM' + suffix)
df16 = pd.read_csv(path_files_2024 + 'Aug-Dec 2024 Havas Meta' + suffix)
df17 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q3 2024 US Data - TikTok' + suffix)
df18 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - TikTok (1)' + suffix)
df19 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - TikTok' + suffix)
df20 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - Google (YouTube)' + suffix)
df21 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - CTV via MiQ (thru 6_30)' + suffix)
df22 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q3 2024 US Data - YouTube via Miq' + suffix)
df23 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Meme' + suffix)
df24 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data -  Oct OOH' + suffix)
df25 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Influencer' + suffix)
df26 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - December CTV via MiQ' + suffix)
df27 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q3 2024 US Data - Uber' + suffix)
df28 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Oct YouTube via MiQ' + suffix)
df29 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Oct TikTok' + suffix)
df30 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - City Marketing' + suffix)
df31 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Pinterest' + suffix)
df32 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q3 2024 US Data - OOH' + suffix)
df33 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q3 2024 US Data - CTV via MiQ' + suffix)
df34 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Nov YouTube via MiQ' + suffix)
df35 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - December YouTube via MiQ' + suffix)
df36 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Nov TikTok' + suffix)
df37 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - December TikTok' + suffix)
df38 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Culture' + suffix)
df39 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - College Ambassador 2024' + suffix)
df40 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - December Snapchat' + suffix)
df41 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Vox' + suffix)
df42 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Nov CTV via MiQ' + suffix)
df43 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - OOH' + suffix)
df44 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Live Nation (OOH)' + suffix)
df45 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - December OOH' + suffix)
df46 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Nov OOH' + suffix)
df47 = pd.read_csv(path_files_2024 + 'Net New 2024 MMM Spend - Net New Items' + suffix)




In [ ]:
df13.to_csv(path_files_2024 + 'US_Bumble_Anything Goes_Dating Exclusivity_2024 AP' + suffix, index=False)
df12.to_csv(path_files_2024 + 'US_Bumble_Audio_Q2 2024' + suffix, index=False)
df11.to_csv(path_files_2024 + 'US_Bumble_Dating Sunday_January 2024' + suffix, index=False)
df10.to_csv(path_files_2024 + 'US_Bumble_SPAN_2024 AP' + suffix, index=False)

In [6]:
def rename_df_columns(df_dict):
    for key in df_dict:
        df = pd.read_csv(df_dict[key] + key + suffix)
        df.columns = ['Channel', 'Country', 'DMA', 'Campaign', 'Quarter', 'Date', 'Impressions', 'Spend']
        
        df.to_csv(df_dict[key] + key + suffix, index=False)
        
    return print('All done!')

rename_df_columns(df_dict)

All done!


In [ ]:

def df_transfos(df_dict):
    df_spend_by_media_quarter = pd.DataFrame(columns=['Channel', 'DMA', 'quarter_year', 'Spend'])
    for key in df_dict:
        df = pd.read_csv(df_dict[key] + key + suffix)
        df.columns = ['Channel', 'Country', 'DMA', 'Campaign', 'Quarter', 'Date', 'Impressions', 'Spend']
        df['source_csv'] = key
        # print(df.head())
        df.Date = df.Date.astype('datetime64[ns]')
        df['quarter_nb'] = df.Date.dt.quarter.astype(str)
        df['year_nb'] = df.Date.dt.year.astype(str)
        df['quarter_year'] = df['year_nb'] + '-' + df['quarter_nb']

        df_spend_temp = df.groupby(['source_csv', 'Channel', 'DMA', 'quarter_year'], dropna=False, as_index = False)['Spend'].sum()

        df_spend_by_media_quarter = pd.concat([df_spend_by_media_quarter, df_spend_temp], ignore_index=True)
        
    return df_spend_by_media_quarter

df_spend_grouped = df_transfos(df_dict)

/var/folders/6s/c8mm3wnn6s51d7sp8cgj057c0000gp/T/ipykernel_30784/3748348050.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_spend_by_media_quarter = pd.concat([df_spend_by_media_quarter, df_spend_temp], ignore_index=True)


In [51]:
df_spend_grouped[df_spend_grouped['Channel']=='Culture'].head()

,Channel,DMA,quarter_year,Spend,source_csv
252,Culture,NaN,2023-1,685000.00,2023_bumble_marketing_spend_editJan24_no_meta_...
253,Culture,NaN,2023-2,356750.00,2023_bumble_marketing_spend_editJan24_no_meta_...
254,Culture,NaN,2023-3,1749389.42,2023_bumble_marketing_spend_editJan24_no_meta_...
255,Culture,NaN,2023-4,4577634.91,2023_bumble_marketing_spend_editJan24_no_meta_...
77579,Culture,Miami,2024-4,524999.00,Restated & Net New MMM Data - Culture


In [49]:
gs = GoogleSheetData()
gs.write_dataframe(df_spend_grouped, "1JjhkNomf5k3GFmrGw0E8Elz2bLyIKrerqTW9Rq0Fke4", "Python_Script_City")

Connected to GSheets
An error occurred: <HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1JjhkNomf5k3GFmrGw0E8Elz2bLyIKrerqTW9Rq0Fke4/values/Python_Script_City%21A1?valueInputOption=USER_ENTERED&alt=json returned "Invalid JSON payload received. Unexpected token.
, ["AO Influencer", NaN, "2023-1", 14722
                    ^">


<HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1JjhkNomf5k3GFmrGw0E8Elz2bLyIKrerqTW9Rq0Fke4/values/Python_Script_City%21A1?valueInputOption=USER_ENTERED&alt=json returned "Invalid JSON payload received. Unexpected token.
, ["AO Influencer", NaN, "2023-1", 14722
                    ^">

In [5]:
df47 = pd.read_csv(path_files_2024 + 'Net New 2024 MMM Spend - Net New Items.csv')

In [17]:
df47

,Channel,Country,DMA,Campaign,Quarter,Date,Impressions,Spend
0,Sports Sponsorships,US,NYC,Liberty Parade,Q4,2024-10-14,0,318099.0
1,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-08-20,0,25000.0
2,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-08-22,0,25000.0
3,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-08-24,0,25000.0
4,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-09-08,0,25000.0
...,...,...,...,...,...,...,...,...
118,College Ambassadors,US,National,Honey Merch,Q4,2024-12-09,0,5732.0
119,College Ambassadors,US,National,Honey Merch,Q4,2024-12-16,0,28258.0
120,College Ambassadors,US,National,Honey Merch,Q4,2024-12-23,0,2211.0
121,Culture,US,Miami,HighSnobiety Art Basel experiential,Q4,2024-12-05,0,375000.0


In [19]:
mapping_dict = {'College Ambassadors': 'College Ambassador'}
df47['Channel'] = df47['Channel'].map(mapping_dict).fillna(df47['Channel'])
df47

,Channel,Country,DMA,Campaign,Quarter,Date,Impressions,Spend
0,Sports Sponsorships,US,NYC,Liberty Parade,Q4,2024-10-14,0,318099.0
1,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-08-20,0,25000.0
2,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-08-22,0,25000.0
3,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-08-24,0,25000.0
4,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-09-08,0,25000.0
...,...,...,...,...,...,...,...,...
118,College Ambassador,US,National,Honey Merch,Q4,2024-12-09,0,5732.0
119,College Ambassador,US,National,Honey Merch,Q4,2024-12-16,0,28258.0
120,College Ambassador,US,National,Honey Merch,Q4,2024-12-23,0,2211.0
121,Culture,US,Miami,HighSnobiety Art Basel experiential,Q4,2024-12-05,0,375000.0


In [10]:
df47.dtypes

Channel         object
Country         object
DMA             object
Campaign        object
Quarter         object
Date            object
Impressions      int64
Spend          float64
dtype: object

In [ ]:
import datetime
df1.Date2 = df1.Date2.astype('datetime64[ns]')
df1['year'] = df1.Date2.dt.year.astype(str)
df_test = df1[df1['year']=='2024']
# df_test['test'] = df_test['Date2'] - datetime.timedelta(days=366)
df_test

In [15]:
df1.columns

Index(['Source', 'Platform', 'Country', 'DMA', 'Campaign', 'Quarter', 'Date',
       'Impressions', ' Spend ', 'Unnamed: 9'],
      dtype='object')

In [16]:
df1.columns = ['Source', 'Channel', 'Country', 'DMA', 'Campaign', 'Quarter', 'Date', 'Impressions', 'Spend', 'Unnamed']

In [17]:
df1.head()

,Source,Channel,Country,DMA,Campaign,Quarter,Date,Impressions,Spend,Unnamed
0,H1,Google,United States,South Bend-Elkhart IN,ISWB,Q1,16/03/2023,"1,817.00",$20.57,NaN
1,H1,Google,United States,South Bend-Elkhart IN,ISWB,Q1,17/03/2023,"1,777.00",$20.01,NaN
2,H1,Google,United States,South Bend-Elkhart IN,ISWB,Q1,18/03/2023,"1,733.00",$20.60,NaN
3,H1,Google,United States,South Bend-Elkhart IN,ISWB,Q1,19/03/2023,"1,723.00",$20.68,NaN
4,H1,Google,United States,South Bend-Elkhart IN,ISWB,Q1,20/03/2023,"1,859.00",$21.05,NaN


In [8]:
def format_single_df(df):
    df.columns = ['Channel', 'Country', 'DMA', 'Campaign', 'Quarter', 'Date', 'Impressions', 'Spend']
    df.Date = df.Date.astype('datetime64[ns]')
    df['Spend'] = df['Spend'].astype("str")
    df['Spend'] = df['Spend'].str.replace('$', '')
    df['Spend'] = df['Spend'].str.replace(',', '')
    df['Spend'] = df['Spend'].str.replace('"', '')
    df['Spend'] = df['Spend'].str.replace(' -', '0')
    df['Spend'] = df['Spend'].astype("float")
    df['Impressions'] = df['Impressions'].astype("str")
    df['Impressions'] = df['Impressions'].str.replace(',', '')
    df['Impressions'] = df['Impressions'].str.replace('"', '')
    df['Impressions'] = df['Impressions'].str.replace('-', '0')
    df['Impressions'] = df['Impressions'].str.replace('.00', '')
    df['Impressions'] = df['Impressions'].str.replace('nan', '0')
    df['Impressions'] = df['Impressions'].fillna('0')
    df['Impressions'] = df['Impressions'].astype("float")
        
    return df

In [9]:
df47_refactored = format_single_df(df47)

In [33]:
df47_refactored.iloc[56]

Channel                                                  Culture
Country                                                       US
DMA                                                     National
Campaign       Special Projects September: NYFW / US Open / R...
Quarter                                                       Q3
Date                                         2024-09-01 00:00:00
Impressions                                                  0.0
Spend                                                    91000.0
Name: 56, dtype: object

In [34]:
df47_refactored['Date_str'] = df47_refactored['Date'].astype(str)
df47_refactored['Date_str'].iloc[51] = '2025-01-02'
df47_refactored['Date_str'].iloc[52] = '2025-01-09'
df47_refactored['Date_str'].iloc[53] = '2025-01-16'
df47_refactored['Date_str'].iloc[54] = '2025-01-23'
df47_refactored['Date_str'].iloc[55] = '2025-01-30'
df47_refactored['Date'] = df47_refactored['Date_str']
df47_refactored = df47_refactored[['Channel', 'Country', 'DMA', 'Campaign', 'Quarter', 'Date', 'Impressions', 'Spend']]

/var/folders/6s/c8mm3wnn6s51d7sp8cgj057c0000gp/T/ipykernel_30784/2820867401.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df47_refactored['Date_str'].iloc[51] = '2025-01-02'
/var/folders/6s/c8mm3wnn6s51d7sp8cgj057c0000gp/T/ipykernel_30

In [18]:
df47_refactored['Date_str'].iloc[51:56] = '2025-01-15'

/var/folders/6s/c8mm3wnn6s51d7sp8cgj057c0000gp/T/ipykernel_30784/1379978459.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df47_refactored['Date_str'].iloc[51:56] = '2025-01-15'
/var/folders/6s/c8mm3wnn6s51d7sp8cgj057c0000gp/T/ipykernel

In [20]:
df47_refactored['Date'] = df47_refactored['Date_str']
df47_refactored = df47_refactored[['Channel', 'Country', 'DMA', 'Campaign', 'Quarter', 'Date', 'Impressions', 'Spend']]

In [35]:
df47_refactored.iloc[50:56]

,Channel,Country,DMA,Campaign,Quarter,Date,Impressions,Spend
50,Culture,US,National,Dating Trends Influencer,Q4,2024-12-30,0.0,5625.0
51,Culture,US,National,Q4 Talent Stunt,Q4,2025-01-02,0.0,131000.0
52,Culture,US,National,Q4 Talent Stunt,Q4,2025-01-09,0.0,131000.0
53,Culture,US,National,Q4 Talent Stunt,Q4,2025-01-16,0.0,131000.0
54,Culture,US,National,Q4 Talent Stunt,Q4,2025-01-23,0.0,131000.0
55,Culture,US,National,Q4 Talent Stunt,Q4,2025-01-30,0.0,131000.0


In [36]:
df47_refactored.Date = df47_refactored.Date.astype('datetime64[ns]')

/var/folders/6s/c8mm3wnn6s51d7sp8cgj057c0000gp/T/ipykernel_30784/115884131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df47_refactored.Date = df47_refactored.Date.astype('datetime64[ns]')


In [ ]:
df47_refactored['Date_str'] = df47_refactored['Date'].astype(str)
df47_refactored['year_nb'] = df47_refactored.Date_parsed.dt.year.astype(str)
df47_refactored[df47_refactored['year_nb']=='2024']

In [ ]:
df1['Spend'] = df1['Spend'].astype("str")
df1['Spend'] = df1['Spend'].str.replace('$', '')
df1['Spend'] = df1['Spend'].str.replace(',', '')
df1['Spend'] = df1['Spend'].str.replace('"', '')
df1['Spend'] = df1['Spend'].str.replace(' -', '0')
df1['Spend'] = df1['Spend'].astype("float")
df1['Impressions'] = df1['Impressions'].astype("str")
df1['Impressions'] = df1['Impressions'].str.replace(',', '')
df1['Impressions'] = df1['Impressions'].str.replace('"', '')
df1['Impressions'] = df1['Impressions'].str.replace('-', '0')
df1['Impressions'] = df1['Impressions'].str.replace('.00', '')
df1['Impressions'] = df1['Impressions'].str.replace('nan', '0')
df1['Impressions'] = df1['Impressions'].fillna('0')
df1['Impressions'] = df1['Impressions'].astype("float")

In [21]:
df1 = df1[['Channel', 'Country', 'DMA', 'Campaign', 'Quarter', 'Date', 'Impressions', 'Spend']]

In [22]:
df3.head()

,EXTEND WHEN ADDING DATA - Source,Country,DMA,Campaign,Quarter,Date,Impressions,Spend
0,College Ambassador,US,Austin,NaN,Q1,2023-02-01,"9,725","2,083.00"
1,College Ambassador,US,Dallas,NaN,Q1,2023-02-01,"14,596","3,026.00"
2,College Ambassador,US,Miami,NaN,Q1,2023-02-01,"10,822","1,649.00"
3,College Ambassador,US,D.C,NaN,Q1,2023-02-01,"22,225","5,139.00"
4,College Ambassador,US,San Francisco,NaN,Q1,2023-02-01,"26,633","4,869.00"


In [25]:
df9['Spend'] = df9['Spend'].astype("str")
df9['Spend'] = df9['Spend'].str.replace('$', '')
df9['Spend'] = df9['Spend'].str.replace(',', '')
df9['Spend'] = df9['Spend'].str.replace('"', '')
df9['Spend'] = df9['Spend'].str.replace(' -', '0')
df9['Spend'] = df9['Spend'].astype("float")
df9['Impressions'] = df9['Impressions'].astype("str")
df9['Impressions'] = df9['Impressions'].str.replace(',', '')
df9['Impressions'] = df9['Impressions'].str.replace('"', '')
df9['Impressions'] = df9['Impressions'].str.replace('-', '0')
df9['Impressions'] = df9['Impressions'].str.replace('.00', '')
df9['Impressions'] = df9['Impressions'].str.replace('nan', '0')
df9['Impressions'] = df9['Impressions'].fillna('0')
df9['Impressions'] = df9['Impressions'].astype("int")

In [ ]:
# Ensure 'Date' is a string
# Ensure 'Date' is a string
df1['Date2'] = df1['Date'].astype(str)

# First, try parsing with dayfirst=True (for DD/MM/YYYY format)
df1['Date_parsed'] = pd.to_datetime(df1['Date'], dayfirst=True, errors='coerce')

# Identify rows that were not parsed (NaT values)
mask = df1['Date_parsed'].isna()

# For those, try parsing with dayfirst=False (for M/D/YY format)
df1.loc[mask, 'Date_parsed'] = pd.to_datetime(df1.loc[mask, 'Date2'], dayfirst=True, errors='coerce')

# Format as YYYY-MM-DD
df1['Date2'] = df1['Date_parsed'].dt.strftime('%Y-%m-%d')

# Drop the helper column
df1.drop(columns=['Date_parsed'], inplace=True)

print(df1[['Date2']].head(50))  # Check the output

In [33]:
df1['Date'] = df1['Date2']

In [43]:
df1.drop(columns=['Date_parsed', 'year_nb'], inplace=True)

In [41]:
df1['Date'] = df1['Date'].str.replace('2024', '2023')

In [42]:
df1['Date_parsed'] = pd.to_datetime(df1['Date'], dayfirst=True, errors='coerce')
df1['year_nb'] = df1.Date_parsed.dt.year.astype(str)
df1[df1['year_nb']=='2024']

/var/folders/6s/c8mm3wnn6s51d7sp8cgj057c0000gp/T/ipykernel_2280/1455866868.py:1: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df1['Date_parsed'] = pd.to_datetime(df1['Date'], dayfirst=True, errors='coerce')


,Channel,Country,DMA,Campaign,Quarter,Date,Impressions,Spend,Date_parsed,year_nb


In [ ]:
df47_refactored.to_csv(path_files_2024 + '2023_bumble_marketing_spend_editJan24_no_meta_college' + suffix, index=False)
# df3.to_csv(path_files_2023 + 'Restated & Net New MMM Data - College Ambassador 2023' + suffix, index=False)
# df9.to_csv(path_files_2023 + '2023 Meta Restated' + suffix, index=False)

In [11]:
df47['Spend'].sum()

np.float64(4626434.0)

In [37]:
df47_refactored.to_csv(path_files_2024 + 'Net New 2024 MMM Spend - Net New Items' + suffix, index=False)

In [52]:
df1_nulldmas = df1[df1['DMA'].isna()]

In [58]:
len(df1_nulldmas.sort_values(by=['Spend'], ascending=False))

1906

In [63]:
df1.loc[537269:]

,Channel,Country,DMA,Campaign,Quarter,Date,Impressions,Spend
537269,Culture,United States,National,Love Letter to Black Women/ Black History Month,Q1,2023-02-17,8200825.0,190000.0
537270,Culture,United States,National,Mental Health Awareness Month,Q2,2023-08-05,1160000.0,25000.0
537271,Culture,United States,National,Mental Health Awareness Month,Q2,2023-05-15,1160000.0,25000.0
537272,Culture,United States,National,Mental Health Awareness Month,Q2,2023-05-22,1160000.0,25000.0
537273,Culture,United States,National,Mental Health Awareness Month,Q2,2023-05-29,1160000.0,25000.0
...,...,...,...,...,...,...,...,...
537407,Culture,United States,National,Alex Cooper Tour,Q4,2023-11-27,480769.0,125000.0
537408,Culture,United States,National,Alex Cooper Tour,Q4,2023-12-04,480769.0,125000.0
537409,Culture,United States,National,Alex Cooper Tour,Q4,2023-12-11,480769.0,125000.0
537410,Culture,United States,National,Alex Cooper Tour,Q4,2023-12-18,480769.0,125000.0


In [62]:
df1.loc[537269:, 'DMA'] = 'National'